In [6]:
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy as np
import pandas as pd
import pickle



from flask import Flask
from flask import request
import requests
from flask import jsonify

import os
import json
from ast import literal_eval
import traceback

application = Flask(__name__)


#загружаем модели из файла
#vec = pickle.load(open("./models/tfidf_multi_class.pickle", "rb"))
#model = lgb.Booster(model_file='./models/lgbm_model_multi_class.txt')


vec = pickle.load(open('./models/feature.pkl', 'rb'))
cv = CountVectorizer(vocabulary=vec)
model = pickle.load(open('./models/clf.sav', 'rb'))

# тестовый вывод
@application.route("/")  
def hello():
    resp = {'message':"Hello World!"}
    
    response = jsonify(resp)
    
    return response

# предикт категории
#{"user_message":"example123rfssg gsfgfd"}
@application.route("/categoryPrediction" , methods=['GET', 'POST'])  
def registration():
    resp = {'message':'ok'
           ,'category': -1
           }

    try:
        getData = request.get_data()
        json_params = json.loads(getData) 
        #print(json_params)
        message = json_params['user_message']
        #print(message)
        #напишите прогноз и верните его в ответе в параметре 'prediction'
        #prediction = model.predict_proba(vec.transform([message]).toarray()).tolist()
        print([message])
        prediction = model.predict_proba(vec.transform([message])).tolist()[0]
        
        print(prediction)
        resp['category'] = prediction
        
    except Exception as e: 
        print(e)
        resp['message'] = e
      
    response = jsonify(resp)
    
    return response #prediction

        

if __name__ == "__main__":
    port = int(os.getenv('PORT', 5000))
    application.run(debug=False, port=port, host='0.0.0.0' , threaded=True)





 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Aug/2020 22:34:04] "GET /categoryPrediction HTTP/1.1" 200 -


['example123rfssg gsfgfd']
[0.8812305762741206, 0.11754282078270196, 0.0012266029431774674]


In [ ]:
pickle.load(open('./models/clf.sav', 'rb'))

# Пример сохранения моделей и прогноза

In [ ]:
#сохраняем векторайзер
import pickle 

labels, levels = pd.factorize(df['category'])
df['category_int'] = labels

texts = df['message']

vec = TfidfVectorizer(max_features=100)
tfidf_mat = vec.fit_transform(texts).toarray()


#vec - модель из прошлой практики

#сохраняем модель в файл
pickle.dump(vec, open("tfidf.pickle", "wb"))

#загружаем модель из файла
vec = pickle.load(open("tfidf.pickle", "rb"))



#### сохраняем бустинг
import lightgbm as lgb
train = pd.DataFrame(tfidf_mat)
train['category_int'] = df['category_int']

train.dropna(inplace = True)

target = 'category_int'
train_columns = list(set(train.columns) - set([target]))


lgb_data_train = lgb.Dataset(train[train_columns], 
                             train[target].values, 
                             free_raw_data=False
                            )
params = {
    'objective':'multiclass', 
    'metric': 'multi_logloss',
    "num_class" : 3,
    'learning_rate': 0.01, 
    'random_state':202008,
    'subsample':0.33,
    'colsample_bytree':0.33,
    'reg_lambda':4
}

model = lgb.train(params, lgb_data_train, num_boost_round=100  )

model.save_model('lgbm_model.txt')

model = lgb.Booster(model_file='lgbm_model.txt')

#test predict
message = 'hello i want to payment but it is blocked'
model.predict(vec.transform([message]).toarray())